In [13]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Itertools

In [1]:
import itertools as it

## Built-in tools

In [2]:
# @property
# allows to define setter, getter and deleter for a class field

# old way (very low-level way : using __setattr__, __getattr__ etc)
class Person:
    def __init__(self, name):
        self._name = name

    def get_name(self):
        print('Getting name')
        return self._name

    def set_name(self, value):
        print('Setting name to ' + value)
        self._name = value

    def del_name(self):
        print('Deleting name')
        del self._name

    # Set property to use get_name, set_name
    # and del_name methods
    name = property(get_name, set_name, del_name, 'Name property')

p = Person('Adam')
print(p.name)
p.name = 'John'
del p.name

Getting name
Adam
Setting name to John
Deleting name


In [3]:
# using @property decorator

class Person:
    def __init__(self, name):
        self._name = name

    @property
    def name(self):
        print('Getting name')
        return self._name

    @name.setter
    def name(self, value):
        print('Setting name to ' + value)
        self._name = value

    @name.deleter
    def name(self):
        print('Deleting name')
        del self._name

p = Person('Adam')
print('The name is:', p.name)
p.name = 'John'
del p.name

# don't need to use () around property

Getting name
The name is: Adam
Setting name to John
Deleting name


## Functools

In [26]:
from functools import cached_property, lru_cache, partial, partialmethod, reduce

In [14]:
# @cached_property - caches the output of an expensive class method (that uses only instance data - thus has to be calculated only once)
import time

class Dataset:

    def __init__(self, data):
        self._data = data
    
    @cached_property
    def fun(self):
        time.sleep(3)
        return self._data
    
inst = Dataset(10)
start = time.time()
inst.fun  # fun is applied, but don't need to use ()
time.time() - start
new_start = time.time()
inst.fun  # cache is applied
time.time() - new_start

10

3.0244932174682617

10

0.0379331111907959

In [18]:
# @lru.cache - caches certain number of recent function calls, so don't have to run the func all over again
# args: maxsize - max number of cached argument sets (default 128, unlimited cache if None)
#       typed - if True f(3) and f(3.0) will be stored separately, since the inputs, though the same, have different type, default False

# cache is stored via a dict -> all arguments must be hashable (e.g. list won't work)
# f(a=4, b=5) and f(b=5, a=4) will be stored separately

# LRU means Least Recently Used

#example
@lru_cache
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

[fib(n) for n in range(16)]

fib.cache_info()  # info on the cache and call history so far
fib.cache_clear()  # clear all cache
fib.cache_info()

# lru_cache only works within the same process (values won't be saved after the process is closed); for permanent cache need to use custom tools (maybe simple save result in a file, then read if file exists, see also percache lib on pypi and similar)

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610]

CacheInfo(hits=28, misses=16, maxsize=128, currsize=16)

CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)

```@functools.total_ordering``` : when providing one of __lt__(), __le__(), __gt__(), or __ge__() and __eq__() to the class, it reconstructs the rest (e.g. >, >= etc from just < and =)

In [32]:
# functools.partial : freezes some portion of the arguments
# partial(func, /, *args, **keywords)

def f(x, y):
    return x-y

f_partial = partial(f, y=4)
f_partial(3)

# partial returns a callable object which contains the following fields
f_partial.func
f_partial.args
f_partial.keywords

-1

<function __main__.f(x, y)>

()

{'y': 4}

In [23]:
# functools.partialmethod : like partial but for class methods

class Cell(object):
    def __init__(self):
        self._alive = False

    @property
    def alive(self):
        return self._alive

    def set_state(self, state):
        self._alive = bool(state)

    set_alive = partialmethod(set_state, True)
    set_dead = partialmethod(set_state, False)

c = Cell()
c.alive
c.set_alive()
c.alive

False

True

In [27]:
# function.reduce : apply function of two arguments cumulatively to the items of iterable, from left to right, so as to reduce the iterable to a single value

reduce(lambda x, y: x+y, [1, 2, 3, 4, 5])

15

In [29]:
# functools.wraps : recommended to use when constructing other decorators to automatically adjust new function metadata

from functools import wraps
def my_decorator(f):
    @wraps(f)
    def wrapper(*args, **kwds):
        print('Calling decorated function')
        return f(*args, **kwds)
    return wrapper

@my_decorator
def example():
    """Docstring"""
    print('Called example function')

## Operator

In [34]:
import operator  # contains standard operators in functional form

operator.le(4, 5)

# others are eq, ne, ge, gt, is  and many others, see https://docs.python.org/3/library/operator.html

True

## Collections

In addition to built-in containers ```list, tuple, dict, set``` implements
```namedtuple, deque, Counter, OrderedDict, defaultdict```

In [37]:
from collections import Counter, namedtuple, deque, OrderedDict, defaultdict

Counter

In [49]:
# Counter

data = [1, 2, 2, 1, 3, 3, 3, 4, 3, 4, 5, 6, 5, 4, 3]
ctr = Counter(data)

ctr
ctr[1] = 4  # can modify ctr as dict
ctr

list(ctr.elements())
ctr.most_common(2)

ctr.subtract(Counter([1, 1, 2]))  # subtract one counter from another inplace
ctr

ctr.update([2, 3, 4])  # update counter using new iterable or another counter
ctr

Counter({1: 2, 2: 2, 3: 5, 4: 3, 5: 2, 6: 1})

Counter({1: 4, 2: 2, 3: 5, 4: 3, 5: 2, 6: 1})

[1, 1, 1, 1, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 6]

[(3, 5), (1, 4)]

Counter({1: 2, 2: 1, 3: 5, 4: 3, 5: 2, 6: 1})

Counter({1: 2, 2: 2, 3: 6, 4: 4, 5: 2, 6: 1})

In [50]:
Counter(a=4, b=3)  # can construct counters directly
Counter({'a':4, 'b':5})

Counter({'a': 4, 'b': 3})

Counter({'a': 4, 'b': 5})

In [51]:
ctr.values()

dict_values([2, 2, 6, 4, 2, 1])

In [56]:
c = Counter([1, 2, 3, 2, 1, 3, 4, 3, 4, 2, 5])

# common patterns with counters
sum(c.values())                 # total of all counts
c.clear()                       # reset all counts
list(c)                         # list unique elements
set(c)                          # convert to a set
dict(c)                         # convert to a regular dictionary
c.items()                       # convert to a list of (elem, cnt) pairs
Counter(dict([('a', 4), ('b', 2)]))    # convert from a list of (elem, cnt) pairs

n=2
c.most_common()[:-n-1:-1]       # n least common elements
c += Counter()                  # remove zero and negative counts

11

[]

set()

{}

dict_items([])

Counter({'a': 4, 'b': 2})

[]

In [57]:
# multiset operations

c = Counter(a=3, b=1)
d = Counter(a=1, b=2)

c+d
c-d  # counts truncated at zero
c&d
c|d

# Counter was designed for counting, but values can be any objects, not necessarily integers

Counter({'a': 4, 'b': 3})

Counter({'a': 2})

Counter({'a': 1, 'b': 1})

Counter({'a': 3, 'b': 2})

defaultdict

In [69]:
def default():  # can be any function w/o arguments
    return 4

d = defaultdict(default)
d['a'] = 9
d

d['a']
d['b']  # for a missing key, an entry with default value is automatically added
d

defaultdict(<function __main__.default()>, {'a': 9})

9

4

defaultdict(<function __main__.default()>, {'a': 9, 'b': 4})

namedtuple

In [80]:
Point = namedtuple('Point', ['x', 'y'])  # fast way to create simple structs
# namedtuple returns a new tuple subclass called Point (like the first arg)

pt = Point(4, 5)

pt.x
pt.y
pt[1]  # standard tuple functionality is all still there

# for short var names can simply pass a single str, like this
Point = namedtuple('Point', 'x y')

d = dict(x=4, y=12)
Point(**d)  # dict to namedtuple

4

5

5

Point(x=4, y=12)

OrderedDict

In [86]:
# like dict by remembers the order of insertion
ord_dict = OrderedDict()
ord_dict['a'] = 4
ord_dict['b'] = 5
ord_dict['c'] = 7

ord_dict
ord_dict.popitem()  # get and remove the last inserted item (LIFO)
ord_dict
ord_dict.popitem(last=False)  # same with the first (FIFO) if last=False
ord_dict

OrderedDict([('a', 4), ('b', 5), ('c', 7)])

('c', 7)

OrderedDict([('a', 4), ('b', 5)])

('a', 4)

OrderedDict([('b', 5)])

collections provide various ABCs like Iterable, Iterator, Container etc (similar may be available in typing - for type check; those from collections should be used for creating concrete subclasses)

## Typing

In [89]:
# type hints
def f(x: str, y: bool) -> int:
    if y:
        return len(x)
    else:
        return 0

Type hints:
* can be added for critical components and ignored for private functions
* will not change the code behaviour
* useful in small scripts that will be only used once
* valuable in the shared, complex or production code that will be maintained later
* tools such as mypy help to show hints related to typing

A rule of thumb is that type hint is necessary when a unit test is necessary for a function; another criteria is if the function is public or private (starts with _ and is only for internal class/module use)

**typing** module provides type aliases for type hints and checks

In [90]:
# provides types for general objects
from typing import List, Tuple, Dict, Sequence, Callable, Iterable, Iterator

# and ways to combine them
from typing import Union, Optional

# can do e.g.
List[str]
Dict[str, int]
Sequence[float]
Union[List[float], float]

# many more are available here https://docs.python.org/3/library/typing.html
# it's often possible to match exactly the needed type

typing.List[str]

typing.Dict[str, int]

typing.Sequence[float]

typing.Union[typing.List[float], float]

In [94]:
from typing import SupportsInt, Literal

isinstance(4, SupportsInt)  # check if there's int() method

Currency = Literal['USD', 'GBP', 'EUR']  # type for restricted list of strings (not any string)
Currency

True

typing.Literal['USD', 'GBP', 'EUR']

## Special methods

like ```__init__``` or ```__add__``` - they have special meaning inside a class

### string representations

* ```__repr__``` - display on console or repr()
* ```__str__``` - str()

### maths operators
*  ```__add__, __sub__, __mul__, __truediv__, __pow__```
* ```__and__, __or__, __xor__``` etc, - logical operators

### container-like class
* ```__len__``` - len()
* ```__getitem__``` - x[item]
* ```__setitem__``` - x[item] = value
* ```__contains__``` - item in x
* ```__iter__``` - iter(), get a new iterator from the container, e.g. in the for loop (for item in x: ...)
* ```__next__``` - next step in an iterator object

### more
* ```__bytes__``` - computes bytestring representation, must return bytestring
* ```__lt__, __eq__, ```etc - comparisons (mb used together with @total_ordering)
* ```__hash__``` - hash map 
* ```__getattr__, __getattribute__``` - getting attribute, ```__getattr__``` is applied after ```__getattribute__``` fails or throws ```AttributeError``` (so that dot-access can be used inside getattr, would lead to an inf loop in certain cases otherwise)
* ```__setattr__, __delattr__``` - set or delete attribute, x.item = value; del x.item
* ```__call__``` - to emulate a callable object
* ```__enter__, __exit__``` - context managers (low-level definition, better to use @contextmanager)



## Inheritance from basic types. Using abc

very good article : https://treyhunner.com/2019/04/why-you-shouldnt-inherit-from-list-and-dict-in-python/
work through and create some examples

In [96]:
# set is iterable, save lists of str in a set if each element is supposed to be unique

for s in {'a', 'b', 'c'}:
    print(s)

b
c
a


In [97]:
import pandas as pd

In [100]:
x = pd.DataFrame([[1, 2, 3], [4, 3, 5]]).T

In [101]:
x

,0,1
0,1,4
1,2,3
2,3,5


In [104]:
x.index.intersection({1, 2})

Int64Index([1, 2], dtype='int64')